In [ ]:
import tensorflow as tf
from tensorflow.keras.utils import image_dataset_from_directory
from tensorflow.keras.layers import Rescaling, MaxPooling2D, Conv2D, Flatten, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras import optimizers
from tensorflow.keras.metrics import top_k_categorical_accuracy
import os

# dataset

In [ ]:
_URL = 'https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip'
path_to_zip = tf.keras.utils.get_file('cats_and_dogs.zip', origin=_URL, extract=True)
PATH = os.path.join(os.path.dirname(path_to_zip), 'cats_and_dogs_filtered')
train_dir =  "/root/.keras/datasets/cats_and_dogs_filtered/train"
validation_dir = "/root/.keras/datasets/cats_and_dogs_filtered/validation"

# tf.data

In [ ]:
train_ds = image_dataset_from_directory(
    train_dir,
    seed=123,
    image_size=(150, 150),
    batch_size=32,
)

val_ds = image_dataset_from_directory(
    validation_dir,
    seed=123,
    image_size=(150, 150),
    batch_size=32,
)

class_names = train_ds.class_names
print(class_names)

Found 2000 files belonging to 2 classes.
Found 1000 files belonging to 2 classes.
['cats', 'dogs']


In [ ]:
AUTOTUNE = tf.data.AUTOTUNE
normalize = Rescaling(1./255)

train_ds = train_ds.map(lambda image, label: (normalize(image), label))
train_ds = train_ds.shuffle(700).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.map(lambda image, label: (normalize(image), label))
val_ds   = val_ds.shuffle(700).prefetch(buffer_size=AUTOTUNE)

# model

In [ ]:
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu',input_shape=(150, 150, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

def top_5_accuracy(in_gt, in_pred):
    return top_k_categorical_accuracy(in_gt, in_pred, k=5)

model.compile(loss='binary_crossentropy', optimizer=optimizers.Adam(0.0001), metrics=['acc', top_5_accuracy])

In [ ]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_20 (Conv2D)          (None, 148, 148, 32)      896       
                                                                 
 max_pooling2d_20 (MaxPoolin  (None, 74, 74, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_21 (Conv2D)          (None, 72, 72, 64)        18496     
                                                                 
 max_pooling2d_21 (MaxPoolin  (None, 36, 36, 64)       0         
 g2D)                                                            
                                                                 
 conv2d_22 (Conv2D)          (None, 34, 34, 128)       73856     
                                                                 
 max_pooling2d_22 (MaxPoolin  (None, 17, 17, 128)     

In [ ]:
history = model.fit(
      train_ds,
      epochs=30,
      validation_data=val_ds
    )

Epoch 1/30
63/63 [==============================] - 97s 1s/step - loss: 0.6959 - acc: 0.5030 - top_5_accuracy: 1.0000 - val_loss: 0.6902 - val_acc: 0.5260 - val_top_5_accuracy: 1.0000
Epoch 2/30
63/63 [==============================] - 97s 2s/step - loss: 0.6826 - acc: 0.5735 - top_5_accuracy: 1.0000 - val_loss: 0.6666 - val_acc: 0.6150 - val_top_5_accuracy: 1.0000
Epoch 3/30
63/63 [==============================] - 95s 1s/step - loss: 0.6619 - acc: 0.6025 - top_5_accuracy: 1.0000 - val_loss: 0.6577 - val_acc: 0.6400 - val_top_5_accuracy: 1.0000
Epoch 4/30
63/63 [==============================] - 97s 2s/step - loss: 0.6161 - acc: 0.6610 - top_5_accuracy: 1.0000 - val_loss: 0.6238 - val_acc: 0.6500 - val_top_5_accuracy: 1.0000
Epoch 5/30
63/63 [==============================] - 95s 1s/step - loss: 0.5846 - acc: 0.6980 - top_5_accuracy: 1.0000 - val_loss: 0.6020 - val_acc: 0.6500 - val_top_5_accuracy: 1.0000
Epoch 6/30
63/63 [==============================] - 95s 1s/step - loss: 0.5331 -

# Note: the model fitted on CPU